In [1]:
import keras
from keras import layers


2024-04-16 01:13:26.743828: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 01:13:26.748177: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 01:13:26.797923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 01:13:27.735895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score, classification_report, confusion_matrix

In [3]:
train=pd.read_csv("/home/ab1/Desktop/SPH6004/Assignment2/Timestep5_train_cat1.csv")
test=pd.read_csv("/home/ab1/Desktop/SPH6004/Assignment2/Timestep5_test_cat1.csv")


In [4]:
train.columns


Index(['id', 'charttime', 'aniongap', 'bicarbonate', 'bun', 'calcium',
       'chloride', 'creatinine', 'glucose', 'sodium', 'potassium',
       'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc',
       'rdw', 'wbc', 'inr', 'pt', 'ptt', 'hosp_admittime', 'hosp_dischtime',
       'icu_intime', 'icu_outtime', 'los_icu', 'icu_death', 'gender', 'race',
       'admission_age', 'weight_admit', 'height', 'charlson_score',
       'atrial_fibrillation', 'malignant_cancer', 'chf', 'ckd', 'cld', 'copd',
       'diabetes', 'hypertension', 'ihd', 'stroke', 'race_encode_African',
       'race_encode_Asian', 'race_encode_Caucasian', 'race_encode_Hispanic',
       'race_encode_Not Specified', 'race_encode_South American',
       'admission_type_DIRECT EMER.', 'admission_type_DIRECT OBSERVATION',
       'admission_type_ELECTIVE', 'admission_type_EU OBSERVATION',
       'admission_type_EW EMER.', 'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', '

In [5]:
# train=pd.read_csv('/home/ab1/Desktop/SPH6004/Assignment2/train5.csv')
# test= pd.read_csv('/home/ab1/Desktop/SPH6004/Assignment2/test5.csv')


train['id'] = train['id'].astype(str)
test['id'] = test['id'].astype(str)

In [6]:
x_train=train.drop(['los_icu','id','charttime',"icu_death","icu_intime","icu_outtime",'hosp_admittime', 'hosp_dischtime',"hour","race","icu_cat"], axis=1)
y_train=train['los_icu']
y_train = y_train.iloc[4::5]


x_test=test.drop(['los_icu','id','charttime',"icu_death","icu_intime","icu_outtime",'hosp_admittime', 'hosp_dischtime',"hour","race","icu_cat"], axis=1)
y_test=test['los_icu']
y_test = y_test.iloc[4::5]




In [7]:
x_test

,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,...,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,14,21.0,23,8.5,104,1.0,91,136.0,3.3,22.8,...,0,0,0,0,1,0,0,0,0,0
1,12,18.0,22,7.9,107,1.1,104,134.0,3.4,26.3,...,0,0,0,0,1,0,0,0,0,0
2,13,18.0,24,8.1,107,1.0,116,134.0,3.8,24.9,...,0,0,0,0,1,0,0,0,0,0
3,13,18.0,27,7.9,112,1.1,119,140.0,3.4,18.3,...,0,0,0,0,1,0,0,0,0,0
4,21,9.0,27,7.3,109,1.4,104,135.0,4.4,30.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5960,13,24.0,13,8.9,107,1.2,90,144.0,5.2,28.0,...,0,0,0,0,0,1,0,0,0,0
5961,13,24.0,13,8.9,107,1.2,90,144.0,5.2,28.0,...,0,0,0,0,0,1,0,0,0,0
5962,13,24.0,13,8.9,107,1.2,90,144.0,5.2,28.0,...,0,0,0,0,0,1,0,0,0,0
5963,13,24.0,13,8.9,107,1.2,90,144.0,5.2,28.0,...,0,0,0,0,0,1,0,0,0,0


In [8]:

num_cols = x_train.columns[x_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
x_test[num_cols] = scaler.fit_transform(x_test[num_cols])

In [9]:
y_train

4        2.78
9        4.00
14       2.54
19       3.49
24       3.18
         ... 
23824    4.17
23829    3.06
23834    3.66
23839    3.13
23844    3.07
Name: los_icu, Length: 4769, dtype: float64

In [10]:

x_train=x_train.values
y_train=y_train.values
x_test=x_test.values
y_test=y_test.values

x_train = x_train.reshape(((x_train.shape[0]//5), 5, x_train.shape[1]))
x_test = x_test.reshape(((x_test.shape[0]//5), 5, x_test.shape[1]))

n_classes = len(np.unique(y_train))

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [11]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res



In [12]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)

In [13]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=7,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)



2024-04-16 01:13:36.887673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-16 01:13:36.888707: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5, 59)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 59)     │    244,795 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 5, 59)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 5, 59)     │        118 │ dropout_1[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 5, 59)     │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 5, 7)      │        420 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 5, 7)      │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 5, 59)     │        472 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 59)     │        118 │ conv1d_1[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 5, 59)     │          0 │ layer_normalizat… │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 59)     │    244,795 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 5, 59)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 59)     │        118 │ dropout_4[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 5, 59)     │          0 │ layer_normalizat… │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 5, 7)      │        420 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 5, 7)      │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 5, 59)     │        472 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 59)     │        118 │ conv1d_3[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 5, 59)     │          0 │ layer_normalizat… │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 991,501 (3.78 MB)

 Trainable params: 991,501 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 18s 115ms/step - loss: 4.4173 - val_loss: 1.0743
Epoch 2/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - loss: 1.5276 - val_loss: 0.7955
Epoch 3/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - loss: 1.1901 - val_loss: 0.8950
Epoch 4/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - loss: 1.1601 - val_loss: 0.8467
Epoch 5/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 109ms/step - loss: 1.0767 - val_loss: 0.7140
Epoch 6/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.9638 - val_loss: 0.7546
Epoch 7/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.9242 - val_loss: 0.6299
Epoch 8/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - loss: 0.8956 - val_loss: 0.6948
Epoch 9/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - loss: 0.9216 - val_loss: 0.6077
Epoch 10/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.8231 - val_loss: 0.6788
Epoch 11/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - loss: 0.8528 - val_loss: 0.6188
Epoch 12/150
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 70m

In [14]:
loss_test=model.evaluate(x_test, y_test)


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6014


In [15]:
loss_test


0.5899181365966797

In [ ]:
model.metrics_names

In [ ]:
x_ho=holdout.drop(['los_icu','id','charttime',"icu_death","icu_intime","icu_outtime",'hosp_admittime', 'hosp_dischtime',"hour","race","icu_cat"], axis=1)
y_ho=holdout['los_icu']
y_ho = y_ho.iloc[4::5]

x_ho[num_cols] = scaler.fit_transform(x_ho[num_cols])


x_ho=x_ho.values
y_ho=y_ho.values

x_ho = x_ho.reshape(((x_ho.shape[0]//5), 5, x_ho.shape[1]))
y_ho[y_ho == -1] = 0

In [ ]:
loss_ho=model.evaluate(x_ho, y_ho)

In [ ]:
loss_ho